In [ ]:
#wget http://koffidb.org/download/results -O KOFFI-data.csv
#wget http://dx.doi.org/10.6019/CHEMBL3885741 -O KFDD-data.xlsx - check this file download

In [58]:
import pandas as pd
import numpy as np
import pyarrow as py

In [2]:
koffi = pd.read_csv('/Users/damien/Projects/drug-binding-kinetics/data/raw/KOFFI-data.csv')

In [3]:
koffi.head()

,ID,article,method,device,Chip/Biosensor,Software,Evaluation Method,Fitting Model,Fixed Partner,Fixed Partner.1,...,comment,Figures and tables,Is raw data present?,Is the raw data of good quality?,Are fits shown?,How do the fits look?,article__doi,article__journal,article__pubdate,article__title
0,309,PMC3982047,Microscale Thermophoresis (MST),Monolith NT.115,NaN,NaN,NaN,NaN,AtNRT1.1,Arabidopsis thaliana nitrate transporter,...,NaN,subfig2,Y,N,Y,B,10.1038/nature13116,Nature,2014-02-26,Molecular basis of nitrate uptake by the plant...
1,1759,PMC4309432,Microscale Thermophoresis (MST),Monolith NT.115,Monolith NT Standard Treated Capillaries,NaN,NaN,NaN,rACBP3,recombinant acyl-CoA-binding protein 3,...,NaN,fig 9,Y,X,Y,R,10.1111/tpj.12692,The Plant Journal,2014-10-06,Arabidopsis acyl-CoA-binding protein ACBP3 par...
2,1761,PMC4309432,Microscale Thermophoresis (MST),Monolith NT.115,Monolith NT Standard Treated Capillaries,NaN,NaN,NaN,rACBP3,recombinant acyl-CoA-binding protein 3,...,NaN,fig 9,Y,X,Y,R,10.1111/tpj.12692,The Plant Journal,2014-10-06,Arabidopsis acyl-CoA-binding protein ACBP3 par...
3,1760,PMC4309432,Microscale Thermophoresis (MST),Monolith NT.115,Monolith NT Standard Treated Capillaries,NaN,NaN,NaN,rACBP3,recombinant acyl-CoA-binding protein 3,...,NaN,fig 9,Y,N,Y,R,10.1111/tpj.12692,The Plant Journal,2014-10-06,Arabidopsis acyl-CoA-binding protein ACBP3 par...
4,307,PMC3982047,Microscale Thermophoresis (MST),Monolith NT.115,NaN,NaN,NaN,NaN,AtNRT1.1,Arabidopsis thaliana nitrate transporter,...,NaN,"fig1a,subfig2",Y,Y,Y,R,10.1038/nature13116,Nature,2014-02-26,Molecular basis of nitrate uptake by the plant...


In [4]:
print('data shape: ', koffi.shape)
print('data columns: ', koffi.columns)

data shape:  (1705, 38)
data columns:  Index(['ID', 'article', 'method', 'device', 'Chip/Biosensor', 'Software',
       'Evaluation Method', 'Fitting Model', 'Fixed Partner',
       'Fixed Partner.1', 'Fixed Partner.2', 'Fixed Partner Type:',
       'Fixed Partner Main Type:', 'Fixed Protein/DNA Sequence or SMILE',
       'Fixed Partner Species', 'Soluble Partner', 'Soluble Partner.1',
       'Soluble Partner.2', 'Soluble Partner Type:',
       'Soluble Partner Main Type:', 'Soluble Protein/DNA Sequence or SMILE',
       'Soluble Partner Species', 'kon/kass/ka', 'STD kon', 'koff/kdis/kd',
       'STD koff', 'KD', 'STD KD', 'comment', 'Figures and tables',
       'Is raw data present?', 'Is the raw data of good quality?',
       'Are fits shown?', 'How do the fits look?', 'article__doi',
       'article__journal', 'article__pubdate', 'article__title'],
      dtype='object')


In [5]:
keep_cols = ['ID', 'method', 'Evaluation Method', 'Fitting Model', 'Fixed Partner',
       'Fixed Partner.1', 'Fixed Partner.2', 'Fixed Partner Type:',
       'Fixed Partner Main Type:',
       'Fixed Partner Species', 'Soluble Partner', 'Soluble Partner.1',
       'Soluble Partner.2', 'Soluble Partner Type:',
       'Soluble Partner Main Type:',
       'Soluble Partner Species', 'kon/kass/ka', 'koff/kdis/kd',
       'KD', 'Is raw data present?', 'Is the raw data of good quality?',
       'Are fits shown?', 'How do the fits look?']

koffi = koffi[keep_cols]

In [6]:
koffi.isnull().sum()

ID                                     0
method                                 7
Evaluation Method                   1101
Fitting Model                        753
Fixed Partner                         17
Fixed Partner.1                      504
Fixed Partner.2                     1005
Fixed Partner Type:                  187
Fixed Partner Main Type:               0
Fixed Partner Species               1056
Soluble Partner                       27
Soluble Partner.1                    617
Soluble Partner.2                   1137
Soluble Partner Type:                217
Soluble Partner Main Type:             0
Soluble Partner Species             1086
kon/kass/ka                          676
koff/kdis/kd                         677
KD                                   182
Is raw data present?                   0
Is the raw data of good quality?       0
Are fits shown?                        0
How do the fits look?                  0
dtype: int64

In [42]:
# dropping rows with no kinetics or binding data, or where one or both binding partners are not listed

drop_rows = koffi.loc[
    koffi['koff/kdis/kd'].isnull() | 
    koffi['kon/kass/ka'].isnull() | 
    koffi['KD'].isnull() | 
    koffi['Fixed Partner'].isnull()| 
    koffi['Soluble Partner'].isnull()
    ].index

koffi.drop(drop_rows, axis=0, inplace=True) 

In [43]:
koffi.isnull().sum()

ID                                    0
method                                0
Evaluation Method                   599
Fitting Model                       366
Fixed Partner                         0
Fixed Partner.1                     295
Fixed Partner.2                     570
Fixed Partner Type:                  99
Fixed Partner Main Type:              0
Fixed Partner Species               534
Soluble Partner                       0
Soluble Partner.1                   355
Soluble Partner.2                   651
Soluble Partner Type:                96
Soluble Partner Main Type:            0
Soluble Partner Species             533
kon/kass/ka                           0
koff/kdis/kd                          0
KD                                    0
Is raw data present?                  0
Is the raw data of good quality?      0
Are fits shown?                       0
How do the fits look?                 0
residence_time                        0
dtype: int64

In [9]:
# creating residence time column from inverse of koff
koffi['residence_time'] = 1 / koffi['koff/kdis/kd']

In [48]:
# creating subset of data where raw data is high quality and binding fits are shown
koffi_hq_subset = koffi.loc[(koffi['Is the raw data of good quality?']=='Y') & (koffi['Are fits shown?']=='Y')]

In [49]:
koffi_hq_subset.columns

Index(['ID', 'method', 'Evaluation Method', 'Fitting Model', 'Fixed Partner',
       'Fixed Partner.1', 'Fixed Partner.2', 'Fixed Partner Type:',
       'Fixed Partner Main Type:', 'Fixed Partner Species', 'Soluble Partner',
       'Soluble Partner.1', 'Soluble Partner.2', 'Soluble Partner Type:',
       'Soluble Partner Main Type:', 'Soluble Partner Species', 'kon/kass/ka',
       'koff/kdis/kd', 'KD', 'Is raw data present?',
       'Is the raw data of good quality?', 'Are fits shown?',
       'How do the fits look?', 'residence_time'],
      dtype='object')

In [50]:
koffi_hq_subset.isnull().sum()

ID                                    0
method                                0
Evaluation Method                    86
Fitting Model                        34
Fixed Partner                         0
Fixed Partner.1                      73
Fixed Partner.2                     145
Fixed Partner Type:                  17
Fixed Partner Main Type:              0
Fixed Partner Species               119
Soluble Partner                       0
Soluble Partner.1                    98
Soluble Partner.2                   174
Soluble Partner Type:                11
Soluble Partner Main Type:            0
Soluble Partner Species              99
kon/kass/ka                           0
koff/kdis/kd                          0
KD                                    0
Is raw data present?                  0
Is the raw data of good quality?      0
Are fits shown?                       0
How do the fits look?                 0
residence_time                        0
dtype: int64

In [51]:
keep_cols = ['ID', 'method', 'Fixed Partner',
       'Fixed Partner Type:',
       'Fixed Partner Main Type:', 'Fixed Partner Species', 'Soluble Partner',
       'Soluble Partner Type:',
       'Soluble Partner Main Type:', 'Soluble Partner Species', 'kon/kass/ka',
       'koff/kdis/kd', 'KD', 'residence_time'
]

In [54]:
koffi = koffi[keep_cols]
koffi_hq_subset = koffi_hq_subset[keep_cols]

In [57]:
print(koffi.shape)
print(koffi_hq_subset.shape)

(957, 14)
(239, 14)


In [59]:
koffi.to_parquet('/Users/damien/Projects/drug-binding-kinetics/data/interim/koffi.parquet')
koffi_hq_subset.to_parquet('/Users/damien/Projects/drug-binding-kinetics/data/interim/koffi_hq_subset.parquet')